In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/cost2100'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cost2100/DATA_Hvalout.mat
/kaggle/input/cost2100/DATA_Htrainin.mat
/kaggle/input/cost2100/DATA_HtestFout_all.mat
/kaggle/input/cost2100/DATA_Htestout.mat
/kaggle/input/cost2100/DATA_Htestin.mat
/kaggle/input/cost2100/DATA_Hvalin.mat
/kaggle/input/cost2100/DATA_Htrainout.mat
/kaggle/input/cost2100/DATA_HtestFin_all.mat


In [9]:
from datetime import datetime
import sys
import traceback
from datetime import datetime
import sys
import traceback
import scipy.io as sio
import torch

In [10]:
r""" The proposed BCsiNet
"""

from torch import nn
from collections import OrderedDict

#from utils import logger

__all__ = ["bcsinet"]


def conv3x3_bn(in_channels, out_channel, stride=1, groups=1):
    r""" 3x3 convolution with padding, followed by batch normalization
    """

    return nn.Sequential(OrderedDict([
        ("conv3x3", nn.Conv2d(in_channels, out_channel, kernel_size=3,
                              stride=stride, padding=1, groups=groups, bias=False)),
        ("bn", nn.BatchNorm2d(num_features=out_channel))
    ]))


class RefineBlock(nn.Module):
    def __init__(self):
        super(RefineBlock, self).__init__()
        self.conv1_bn = conv3x3_bn(2, 8)
        self.conv2_bn = conv3x3_bn(8, 16)
        self.conv3_bn = conv3x3_bn(16, 2)
        self.activation = nn.LeakyReLU(negative_slope=0.3, inplace=True)
        self.identity = nn.Identity()

    def forward(self, x):
        identity = self.identity(x)

        residual = self.activation(self.conv1_bn(x))
        residual = self.activation(self.conv2_bn(residual))
        residual = self.conv3_bn(residual)

        return self.activation(residual + identity)


class TinyRefineBlock(nn.Module):
    r"""
    This is headC for BCsiNet. Residual architecture is included.
    """
    def __init__(self):
        super(TinyRefineBlock, self).__init__()
        self.conv1_bn = conv3x3_bn(2, 4)
        self.conv2_bn = conv3x3_bn(4, 2)
        self.relu = nn.LeakyReLU(negative_slope=0.3, inplace=True)
        self.identity = nn.Identity()

    def forward(self, x):
        identity = self.identity(x)

        residual = self.relu(self.conv1_bn(x))
        residual = self.conv2_bn(residual)

        return self.relu(residual + identity)


class BCsiNet(nn.Module):
    def __init__(self, reduction, encoder_head, num_refinenet):
        super(BCsiNet, self).__init__()
        #logger.info(f"=> Model BCsiNet with reduction={reduction}, ")

        in_channels, total_size, w, h = 2, 2048, 32, 32
        if encoder_head == 'A':
            encoder_feature = [
                ("conv3x3_bn", conv3x3_bn(in_channels, 2)),
                ("relu", nn.LeakyReLU(negative_slope=0.3, inplace=True))
            ]
        elif encoder_head == 'B':
            encoder_feature = [
                ("conv3x3_bn1", conv3x3_bn(in_channels, 2)),
                ("relu1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ("conv3x3_bn2", conv3x3_bn(in_channels, 2)),
                ("relu2", nn.LeakyReLU(negative_slope=0.3, inplace=True))
            ]
        elif encoder_head == 'C':
            encoder_feature = [
                ("conv3x3_bn1", conv3x3_bn(in_channels, 2)),
                ("relu1", nn.LeakyReLU(negative_slope=0.3, inplace=True)),
                ("tiny_refine1", TinyRefineBlock())
            ]
        else:
            raise ValueError(f'Illegal encoder type {encoder_head}')
        self.encoder_feature = nn.Sequential(OrderedDict(encoder_feature))
        self.encoder_binary_fc = nn.Linear(total_size, total_size // reduction)

        self.decoder_fc = nn.Linear(total_size // reduction, total_size)
        decoder_feature = []
        for i in range(num_refinenet):
            decoder_feature.append((f"refine{i}", RefineBlock()))
        self.decoder_feature = nn.Sequential(OrderedDict(
            decoder_feature + [
                ("conv3x3_bn", conv3x3_bn(2, in_channels)),
                ("sigmoid", nn.Sigmoid())
            ]))

        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.xavier_uniform_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _fc_binarization(self):
        r"""
        Note that this PyTorch based binarization only proves the correctness of the
        proposed BCsiNet for simplicity. In order to observe the memory saving and
        inference speed up, C++ codes is needed on general CPU while more customized
        codes are required for ASIC chips at resource limited user equipment.
        """
        
        module = self.encoder_binary_fc
        data = module.weight.data
        mn = data.nelement()
        alpha = data.norm(1).div(mn)
        module.weight.data = data.sign().mul(alpha)

    def adding_noise(self,x):
        # Compute signal power
        signal_power = torch.mean(x**2)

        # Define the desired SNR in dB (e.g., 20 dB)
        SNR_dB = 40

        # Compute the SNR in linear scale
        SNR_linear = 10**(SNR_dB / 10)

        # Compute the noise power based on the SNR
        noise_power = signal_power / SNR_linear

        # Generate Gaussian noise with the same shape as the input tensor
        noise = torch.randn_like(x) * torch.sqrt(noise_power)

        # Add the noise to the input tensor
        x_noisy = x + noise

        return x_noisy

    def forward(self, x):
        assert self.training is False, 'This repo works only for inference'
        n, c, h, w = x.detach().size()

        # For encoder inference at UE
        out = self.encoder_feature(x)
        out = self.encoder_binary_fc(out.view(n, -1))
        #out = self.adding_noise(out)

        # For decoder inference at BS
        out = self.decoder_fc(out)
        out = self.decoder_feature(out.view(n, c, h, w))

        return out


def bcsinet(reduction=4, encoder_head='A', num_refinenet=2):
    r""" Create a proposed BCsiNet model.
    """

    model = BCsiNet(reduction=reduction,
                    encoder_head=encoder_head,
                    num_refinenet=num_refinenet)
    return model


In [11]:
model=bcsinet(reduction=4, encoder_head='A', num_refinenet=2)

In [12]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [13]:
pytorch_total_params

2103064

In [14]:
import torch
#from Trans_Net.utils import *


state_dict = torch.load(r'/kaggle/input/bcsinet-savemodels-a2in01/model.pth',map_location=torch.device('cpu'))['state_dict']

/tmp/ipykernel_31/159798352.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(r'/kaggle/input/bcsinet-savemodels-a2in01/model.pth',map_location=tor

In [15]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [16]:
model.eval()

BCsiNet(
  (encoder_feature): Sequential(
    (conv3x3_bn): Sequential(
      (conv3x3): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): LeakyReLU(negative_slope=0.3, inplace=True)
  )
  (encoder_binary_fc): Linear(in_features=2048, out_features=512, bias=True)
  (decoder_fc): Linear(in_features=512, out_features=2048, bias=True)
  (decoder_feature): Sequential(
    (refine0): RefineBlock(
      (conv1_bn): Sequential(
        (conv3x3): Conv2d(2, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2_bn): Sequential(
        (conv3x3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv3_bn)

In [17]:
model.to('cpu')

BCsiNet(
  (encoder_feature): Sequential(
    (conv3x3_bn): Sequential(
      (conv3x3): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): LeakyReLU(negative_slope=0.3, inplace=True)
  )
  (encoder_binary_fc): Linear(in_features=2048, out_features=512, bias=True)
  (decoder_fc): Linear(in_features=512, out_features=2048, bias=True)
  (decoder_feature): Sequential(
    (refine0): RefineBlock(
      (conv1_bn): Sequential(
        (conv3x3): Conv2d(2, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2_bn): Sequential(
        (conv3x3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv3_bn)

In [18]:
criterion = nn.MSELoss().to('cpu')

In [19]:
model._fc_binarization()

In [20]:
envir = 'indoor'  # 'indoor' or 'outdoor'
# image params
img_height = 32
img_width = 32
img_channels = 2
img_total = img_height * img_width * img_channels
# network params
#residual_num = 2
encoded_dim = 512  # compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32

In [25]:
# envir = 'indoor'  # 'indoor' or 'outdoor'
# # image params
# img_height = 32
# img_width = 32
# img_channels = 2
# img_total = img_height * img_width * img_channels
# # network params
# residual_num = 2
# encoded_dim = 512  # compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32
# Data loading
#/kaggle/input/cost2100/DATA_HtestFout_all.mat
if envir == 'indoor':
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htrainin.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Hvalin.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_Htestin.mat')
    x_test = mat['HT']  # array
    mat = sio.loadmat(r'/kaggle/input/cost2100/DATA_HtestFin_all.mat')
    X_test = mat['HF_all']  # array

elif envir == 'outdoor':
    mat = sio.loadmat(r'archive\DATA_Htrainout.mat')
    x_train = mat['HT']  # array
    mat = sio.loadmat(r'archive\DATA_Hvalout.mat')
    x_val = mat['HT']  # array
    mat = sio.loadmat(r'archive\DATA_Htestout.mat')
    x_test = mat['HT']  # array
    mat = sio.loadmat(r'archive\DATA_HtestFout_all.mat')
    X_test = mat['HF_all']  # array

In [26]:
import numpy as np

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

x_train_length = len(x_train)
x_val_length = len(x_val)
x_test_length = len(x_test)

x_train = np.reshape(x_train, (x_train_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_val = np.reshape(x_val, (x_val_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (x_test_length, img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format

x_train = torch.tensor(x_train)
x_val = torch.tensor(x_val)
x_test = torch.tensor(x_test)

In [27]:
import math
with torch.no_grad():

    #torch.cuda.empty_cache()
    model.eval()
    device='cpu'
    x_hat = model(x_test)
    #torch.quantization.convert(quantized_model, inplace=True)
    x_test = x_test.to('cpu')
    x_hat=x_hat.to('cpu')

    # Calcaulating the NMSE and rho
    # if envir == 'indoor':
    #     mat = sio.loadmat('D:\Cost2100\DATA_HtestFin_all.mat')
    #     X_test = mat['HF_all']  # array

    # elif envir == 'outdoor':
    #     mat = sio.loadmat('D:\Cost2100\DATA_HtestFout_all.mat')
    #     X_test = mat['HF_all']  # array

    #X_test = torch.tensor(X_test)
    #X_test = torch.reshape(X_test, (len(X_test), img_height, 125))
    x_test_real = torch.reshape(x_test[:, 0, :, :], (len(x_test), -1))
    x_test_imag = torch.reshape(x_test[:, 1, :, :], (len(x_test), -1))
    x_test_C = x_test_real - 0.5 + 1j * (x_test_imag - 0.5)
    x_hat_real = torch.reshape(x_hat[:, 0, :, :], (len(x_hat), -1))
    x_hat_imag = torch.reshape(x_hat[:, 1, :, :], (len(x_hat), -1))
    x_hat_C = x_hat_real - 0.5 + 1j * (x_hat_imag - 0.5)
    x_hat_F = torch.reshape(x_hat_C, (len(x_hat_C), img_height, img_width))
    X_hat = torch.fft.fft(torch.cat((x_hat_F, torch.zeros((len(x_hat_C), img_height, 257 - img_width))), axis=2), axis=2)
    X_hat = X_hat[:, :, 0:125]

    #n1 = torch.sqrt(torch.sum(torch.conj(X_test) * X_test, axis=1))
    #n2 = torch.sqrt(torch.sum(torch.conj(X_hat) * X_hat, axis=1))
    #aa = abs(torch.sum(torch.conj(X_test) * X_hat, axis=1))
    #rho = torch.mean(aa / (n1 * n2), axis=1)
    X_hat = torch.reshape(X_hat, (len(X_hat), -1))
    #X_test = torch.reshape(X_test, (len(X_test), -1))
    power = torch.sum(abs(x_test_C) ** 2, axis=1)
    power_d = torch.sum(abs(X_hat) ** 2, axis=1)
    mse = torch.sum(abs(x_test_C - x_hat_C) ** 2, axis=1)
    NMSE = 10 * math.log10(torch.mean(mse / power))
    #Correlation = torch.mean(rho).item().real

    # print("In " + envir + " environment")
    # print("When dimension is", encoded_dim)
    print("NMSE is ", NMSE)
    #print("Correlation is ", Correlation)
#
# file = 'CsiNet_' + (envir) + '_dim' + str(encoded_dim) + time.strftime('_%m_%d_%H_%M')
# outfile = "result/result_%s.mat" % file
# savemat(outfile, {'train_loss_history': train_loss_history,
#                   'val_loss_history': val_loss_history,
#                   'training_time': training_time,
#                   'NMSE': NMSE,
#                   'Correlation': Correlation})

NMSE is  -17.25323241764062
